<a href="https://colab.research.google.com/github/adiratnaa/Data-Communication/blob/main/socket_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Socket Menggunakan Python

Tutorial bagaimana memprogram aplikasi jaringan TCP/IP, dengan contoh web (protokol HTTP).

## Introduction
* Socket : https://docs.python.org/3/library/socket.html
* HTTP : https://fr.wikipedia.org/wiki/Hypertext_Transfer_Protocol
* Tips : http://aurelien.esnard.emi.u-bordeaux.fr/teaching/doku.php?id=rx3:index
* Cours Moodle DIU : https://moodle1.u-bordeaux.fr/course/view.php?id=4713

## Memahami Protokol Web (HTTP)

Dengan Wireshark, verifikasi bahwa kita memahami :

* https://github.com/diu-uf-bordeaux/bloc3/raw/master/reseaux/trace/http.pcap

Info lebih lanjut tentang course Moodle: https://moodle1.u-bordeaux.fr/course/view.php?id=4713


## Request Web

Perhatikan contoh situs web: www.perdu.com

Mari kita install dulu program *netcat*:


In [1]:
! apt-get -q -y install netcat-openbsd

Reading package lists...
Building dependency tree...
Reading state information...
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-11-0 cuda-nsig

Mari buat permintaan HTTP GET secara manual dengan *netcat* tools :

In [2]:
! echo -e "GET / HTTP/1.1\r\nHost: www.perdu.com\r\n\r\n" | netcat -N www.perdu.com 80

HTTP/1.1 200 OK
Date: Thu, 03 Feb 2022 03:13:26 GMT
Server: Apache
Upgrade: h2
Connection: Upgrade
Last-Modified: Thu, 02 Jun 2016 06:01:08 GMT
ETag: "cc-5344555136fe9"
Accept-Ranges: bytes
Content-Length: 204
Cache-Control: max-age=600
Expires: Thu, 03 Feb 2022 03:23:26 GMT
Vary: Accept-Encoding,User-Agent
Content-Type: text/html

<html><head><title>Vous Etes Perdu ?</title></head><body><h1>Perdu sur l'Internet ?</h1><h2>Pas de panique, on va vous aider</h2><strong><pre>    * <----- vous &ecirc;tes ici</pre></strong></body></html>


Harap diperhatikan: protokol HTTP mengharuskan setiap baris diakhiri dengan karakter "\r\n" (CR+LF).

## String Karakter

Hati-hati, fungsi send()/recv() dari modul *socket* tidak menangani *str* tetapi *bytearray*:

* https://docs.python.org/3/library/stdtypes.html#bytearray
* https://docs.python.org/3/library/stdtypes.html#str



In [3]:
s0 = "coucou"           # str object       
b0 = b"coucou"          # bytearray (notez le préfixe b)
b1 = s0.encode()        # convert str to bytearray
s1 = b0.decode()        # convert bytearray to str (utf-8 encoding)
print("s0 =", s0)
print("b0 =", b0)

s0 = coucou
b0 = b'coucou'


*string* yang diformat dengan Python:

* https://docs.python.org/3/tutorial/inputoutput.html

In [4]:
"Hello, my name is {}!".format("Adiratna")

'Hello, my name is Adiratna!'

## Modul Soket di Python3

* https://docs.python.org/3/howto/sockets.html
* https://docs.python.org/3/library/socket.html


In [5]:
import socket

Tidak mudah menemukan situs web yang masih menggunakan HTTP 1.0... Perhatikan pesan kesalahannya!

## Klien HTTP 1.1

Dalam versi HTTP 1.1-nya, bidang **Host** harus ditentukan di header permintaan HTTP. Selain itu, akan berguna untuk menyetel bidang **Connection** ke nilai **close** untuk memaksa penutupan koneksi TCP/IP, di akhir respons. Berikut cara menyiapkan HTTP request ke web server www.perdu.com (port 80):

In [6]:
HOST = b'www.perdu.com'
PORT = 80
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))
request = b'GET / HTTP/1.1\r\n'
request += b'Host: ' + HOST + b'\r\n'
request += b'Connection: close\r\n'
request += b'\r\n'
print(request.decode())


GET / HTTP/1.1
Host: www.perdu.com
Connection: close




Kirim permintaan dan terima 1024 karakter pertama dari tanggapan...

In [7]:
s.sendall(request)
answer = s.recv(1024)
print(answer.decode())
s.close()


HTTP/1.1 200 OK
Date: Thu, 03 Feb 2022 03:13:26 GMT
Server: Apache
Upgrade: h2
Connection: Upgrade, close
Last-Modified: Thu, 02 Jun 2016 06:01:08 GMT
ETag: "cc-5344555136fe9"
Accept-Ranges: bytes
Content-Length: 204
Cache-Control: max-age=600
Expires: Thu, 03 Feb 2022 03:23:26 GMT
Vary: Accept-Encoding,User-Agent
Content-Type: text/html

<html><head><title>Vous Etes Perdu ?</title></head><body><h1>Perdu sur l'Internet ?</h1><h2>Pas de panique, on va vous aider</h2><strong><pre>    * <----- vous &ecirc;tes ici</pre></strong></body></html>



Jika kita ingin menerima respons dengan ukuran variabel, kita harus "melakukan pengulangan" hingga kita menerima respons kosong (karakteristik penutupan koneksi server).

In [8]:
s.sendall(request)

while True:
    answer = s.recv(1024)
    if answer == b'': break
    print(answer.decode('utf-8'), end='')

s.close()


OSError: ignored